# Benchmark Date: 2025-12-10

mbo_utilities loading performance benchmarks

In [ ]:
import mbo_utilities
print(f"mbo_utilities version: {mbo_utilities.__version__}")
print(f"Benchmark date: 2025-12-10")

In [ ]:
from pathlib import Path
import time
import numpy as np
from tifffile import TiffFile
from mbo_utilities.metadata import get_metadata
from mbo_utilities.arrays.tiff import MboRawArray
from mbo_utilities import imread

# test path - network drive
test_path = r"\\rbo-s1\S1_DATA\lbm\kbarber\2025-11-04-mk311\raw\green"

In [ ]:
p = Path(test_path)
files = sorted(list(p.glob("*.tif")) + list(p.glob("*.tiff")))
print(f"Found {len(files)} files")

## Array Initialization

In [ ]:
%%timeit -n 1 -r 3
arr = MboRawArray(files=files)
for tf in arr.tiff_files:
    tf.close()

In [ ]:
arr = MboRawArray(files=files)
print(f"shape: {arr.shape}")
print(f"dtype: {arr.dtype}")
print(f"num_channels (z-planes): {arr.num_channels}")
print(f"num_rois: {arr.num_rois}")
print(f"fix_phase: {arr.fix_phase}")
print(f"use_fft: {arr.use_fft}")

## Indexing Shapes

Verify what each index returns:

In [ ]:
arr.fix_phase = False

print("Indexing patterns:")
print(f"  arr[0].shape     = {arr[0].shape}      <- frame 0, ALL z-planes")
print(f"  arr[0, 0].shape  = {arr[0, 0].shape}   <- frame 0, z-plane 0")
print(f"  arr[:, 0].shape  = {arr[:, 0].shape}  <- ALL frames, z-plane 0")

## Single Frame Access - NO Phase Correction

In [ ]:
arr.fix_phase = False
print(f"fix_phase = {arr.fix_phase}")

In [ ]:
# arr[0, 0] - single frame, single z-plane
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i, 0] (single frame, single z):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

In [ ]:
# arr[0] - single frame, ALL z-planes
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i] (single frame, ALL z-planes):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

## Single Frame Access - WITH Phase Correction

In [ ]:
arr.fix_phase = True
arr.use_fft = False  # integer method
print(f"fix_phase = {arr.fix_phase}, use_fft = {arr.use_fft}")

In [ ]:
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i, 0] WITH phase corr (integer method):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

In [ ]:
arr.use_fft = True  # FFT method
print(f"fix_phase = {arr.fix_phase}, use_fft = {arr.use_fft}")

times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"arr[i, 0] WITH phase corr (FFT method):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

## Direct TiffFile Read (baseline)

In [ ]:
tf = arr.tiff_files[0]

times = []
for i in range(20):
    t0 = time.perf_counter()
    page = tf.asarray(key=i)
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

print(f"Direct tf.asarray(key=i):")
print(f"  Mean: {np.mean(times):.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

## Reduction Methods

In [ ]:
arr.fix_phase = False

# test on subset of frames
n_frames = min(50, arr.num_frames)
print(f"Testing reductions on first {n_frames} frames, z-plane 0")

In [ ]:
# mean
t0 = time.perf_counter()
result = arr[:n_frames, 0].mean(axis=0)
t1 = time.perf_counter()
print(f"arr[:n, 0].mean(axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

In [ ]:
# max projection
t0 = time.perf_counter()
result = arr[:n_frames, 0].max(axis=0)
t1 = time.perf_counter()
print(f"arr[:n, 0].max(axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

In [ ]:
# std
t0 = time.perf_counter()
result = arr[:n_frames, 0].std(axis=0)
t1 = time.perf_counter()
print(f"arr[:n, 0].std(axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

In [ ]:
# percentile
t0 = time.perf_counter()
data = arr[:n_frames, 0]
result = np.percentile(data, 99, axis=0)
t1 = time.perf_counter()
print(f"np.percentile(arr[:n, 0], 99, axis=0): {(t1-t0)*1000:.1f} ms, shape={result.shape}")

## Multi-frame Access

In [ ]:
arr.fix_phase = False

for n in [10, 50, 100]:
    if n > arr.num_frames:
        continue
    times = []
    for _ in range(3):
        t0 = time.perf_counter()
        data = arr[:n, 0]
        t1 = time.perf_counter()
        times.append((t1 - t0) * 1000)
    print(f"arr[:{n}, 0]: {np.mean(times):.1f} ms (shape={data.shape})")

## __getitem__ Breakdown

In [ ]:
from mbo_utilities.util import listify_index, index_length
from mbo_utilities.arrays.tiff import _convert_range_to_slice

arr.fix_phase = False
key = (0, 0)

# step 1: key parsing
t0 = time.perf_counter()
for _ in range(1000):
    if not isinstance(key, tuple):
        k = (key,)
    else:
        k = key
    t_key, z_key, _, _ = tuple(_convert_range_to_slice(x) for x in k) + (slice(None),) * (4 - len(k))
t1 = time.perf_counter()
print(f"Key parsing x1000: {(t1-t0)*1000:.2f} ms")

# step 2: listify_index
t0 = time.perf_counter()
for _ in range(1000):
    frames = listify_index(t_key, arr.num_frames)
    chans = listify_index(z_key, arr.num_channels)
t1 = time.perf_counter()
print(f"listify_index x1000: {(t1-t0)*1000:.2f} ms")

# step 3: process_rois (includes I/O)
t0 = time.perf_counter()
out = arr.process_rois(frames, chans)
t1 = time.perf_counter()
print(f"process_rois x1: {(t1-t0)*1000:.2f} ms")

In [ ]:
# index_length vs len(listify_index)
dim_size = arr.shape[2]  # height

t0 = time.perf_counter()
for _ in range(1000):
    x = len(listify_index(slice(None), dim_size))
t1 = time.perf_counter()
print(f"len(listify_index(slice(None), {dim_size})) x1000: {(t1-t0)*1000:.2f} ms")

t0 = time.perf_counter()
for _ in range(1000):
    x = index_length(slice(None), dim_size)
t1 = time.perf_counter()
print(f"index_length(slice(None), {dim_size}) x1000: {(t1-t0)*1000:.2f} ms")

## Summary

In [ ]:
print("=" * 60)
print("BENCHMARK SUMMARY")
print("=" * 60)
print(f"Date: 2025-12-10")
print(f"mbo_utilities: {mbo_utilities.__version__}")
print(f"Data: {test_path}")
print(f"Shape: {arr.shape}")
print(f"Files: {len(files)}")
print("=" * 60)

In [ ]:
# cleanup
for tf in arr.tiff_files:
    tf.close()